In [3]:
CLIENT_ID = 'P40F20UHNUGBTM0V0251ETOQYTT442BJEJ3KG42O2QL2DSNV' # your Foursquare ID
CLIENT_SECRET = 'CHV223VTL3F5OCLRHGKTVCF313K1ZBYOSSPKOS0UA5ODBIMR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: P40F20UHNUGBTM0V0251ETOQYTT442BJEJ3KG42O2QL2DSNV
CLIENT_SECRET:CHV223VTL3F5OCLRHGKTVCF313K1ZBYOSSPKOS0UA5ODBIMR


In [4]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


### IMPORTING LIBRARIES......

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup 

In [9]:
pip install beautifulsoup4

In [10]:
from bs4 import BeautifulSoup 

### COLLECTING/SCRAPEING TABLE FROM THE WIKIPEDIA PAGE

#### >Acquire data from the Wikipedia table


In [57]:
data_req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup_data = BeautifulSoup(data_req, 'html.parser')

postal_list = []
borough_list = []
hood_list = []

#### >Append the column-lists with the data acquired from the Wikipedia table


In [58]:
for row in soup_data.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postal_list.append(cells[0].text.rstrip('\n'))
        borough_list.append(cells[1].text.rstrip('\n'))
        hood_list.append(cells[2].text.rstrip('\n'))

#### >Generate a new DataFrame object


In [59]:
toronto_df =[]
toronto_df = pd.DataFrame({"PostalCode": postal_list,  "Borough": borough_list, "Neighborhood": hood_list})

#### >Drop rows without assigned borough


In [60]:
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### >Group the Neighborhood based on the Borough (and Postal code)


In [61]:
toronto_df = toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df.head(30)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### > If there is a cell with non-assigned neighborhood, the neighborhood name will use its borough instead. But now the table doesn't contain any non-assigned neighborhood name, so I will comment the code instead


In [64]:
#for index, row in toronto_df.iterrows():
#    if row["Neighborhood"] == "Not assigned":
#        row["Neighborhood"] = row["Borough"]

#### >We can check the number of rows per request


In [66]:
toronto_df.shape


(103, 3)

#### > Read the CSV file which contains geospatial coordinates, and give it a suitable column name


In [69]:
coord_data = pd.read_csv("Geospatial_Coordinates.csv")
coord_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

#### > Merge the main dataframe with the geospatial coordinates dataframe


In [71]:
toronto_df = toronto_df.merge(coord_data, on="PostalCode", how="left")


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [74]:
toronto_df_updated = toronto_df
toronto_df_updated.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### > Now, get the coordinate location for the whole Toronto instead of individual neighborhoods, so that we can generate the map later


In [78]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### > Generate the map of Toronto


In [80]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_updated['Latitude'], toronto_df_updated['Longitude'], toronto_df_updated['Borough'], toronto_df_updated['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### > Make a function to see nearby venues

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### > Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category, via one-hot encoding

In [97]:
manhattan_venues = getNearbyVenues(names=toronto_df_updated['Neighborhood'],
                                   latitudes=toronto_df_updated['Latitude'],
                                   longitudes=toronto_df_updated['Longitude']
                                  )



manhattan_venues.groupby('Neighborhood').count()



# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]



manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

#### > Define function to sort the venues in descending order

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### > Display the top 10 venues for each neighborhood.

In [109]:


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dessert Shop,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Dance Studio,Athletics & Sports,Pool,Skating Rink,Gym,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Pharmacy,Sushi Restaurant,Shopping Mall,Restaurant,Chinese Restaurant,Pizza Place,Ice Cream Shop
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Greek Restaurant,Pharmacy,Pizza Place,Café,Pub


#### > Perform the main k-clustering algorithm

In [107]:
# set number of clusters
kclusters = 7

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

manhattan_merged = toronto_df_updated

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head(500) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Print Shop,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,6.0,Golf Course,Bar,Women's Store,Dessert Shop,Fabric Shop,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Rental Car Location,Electronics Store,Breakfast Spot,Intersection,Bank,Medical Center,Mexican Restaurant,Dim Sum Restaurant,Eastern European Restaurant,Drugstore
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Women's Store,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Fried Chicken Joint,Thai Restaurant,Bakery,Athletics & Sports,Caribbean Restaurant,Bank,Gas Station,Hakka Restaurant,Donut Shop,Dog Run
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Convenience Store,Women's Store,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,2.0,Train Station,Convenience Store,Discount Store,Chinese Restaurant,Department Store,Bus Station,Coffee Shop,Donut Shop,Drugstore,Dog Run
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,2.0,Bakery,Bus Line,Park,Metro Station,Intersection,Bus Station,Ice Cream Shop,Soccer Field,Coworking Space,Creperie
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,2.0,Intersection,American Restaurant,Movie Theater,Motel,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2.0,College Stadium,Skating Rink,General Entertainment,Café,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


#### Finally, we can visualize the k-clustering made above into the map of Toronto city

In [112]:
import math

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if math.isnan(cluster) == True:
        cluster = kclusters
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters